# 1. Import data

In [1]:
import pandas as pd
import numpy as np
import pyomo.environ as pyo
from pyomo.environ import *
from pyomo.opt import SolverFactory
from pyomo.opt import SolverStatus, TerminationCondition

In [2]:
data = pd.read_excel('data.xlsx')

In [3]:
data

,index,sector,from,to,flight,DOW_ETD,ETD,DOW_ETA,ETA,AC,revenue,cost
0,BKKSGNVN600A3211,BKKSGN,BKK,SGN,VN600,1,4.333333,1,6.166667,A321,349.09,290.57
1,BKKSGNVN600A3212,BKKSGN,BKK,SGN,VN600,2,4.333333,2,6.166667,A321,349.09,290.57
2,BKKSGNVN600A3213,BKKSGN,BKK,SGN,VN600,3,4.333333,3,6.166667,A321,349.09,290.57
3,BKKSGNVN600A3214,BKKSGN,BKK,SGN,VN600,4,4.333333,4,6.000000,A321,349.09,290.57
4,BKKSGNVN600A3215,BKKSGN,BKK,SGN,VN600,5,4.333333,5,6.000000,A321,349.09,290.57
...,...,...,...,...,...,...,...,...,...,...,...,...
8124,VTEPNHVN921B7874,VTEPNH,VTE,PNH,VN921,4,5.166667,4,6.583333,B787,535.41,445.50
8125,VTEPNHVN921B7875,VTEPNH,VTE,PNH,VN921,5,5.166667,5,6.583333,B787,535.41,445.50
8126,VTEPNHVN921B7876,VTEPNH,VTE,PNH,VN921,6,5.166667,6,6.583333,B787,535.41,445.50
8127,VTEPNHVN921B7877,VTEPNH,VTE,PNH,VN921,7,5.416667,7,6.666667,B787,535.41,445.50


In [4]:
# tạo dictionary về số tàu bay đã thuê mua:
fleet = {'A350':14, 'B787':17, 'A321':62}

# 2. Sets

In [5]:
model = pyo.ConcreteModel()

In [6]:
# create set of sector
# note: lam the nao de biet chuyen chieu di nao tuong ung voi chuyen chieu ve? co can tuong ung ko?
model.sector = pyo.Set(initialize = data['sector'].unique())
sector = model.sector

In [7]:
# create set of aircraft type
model.ac_type = pyo.Set(initialize = data['AC'].unique())
ac_type = model.ac_type

In [8]:
# create set of flight no
model.flight_no = pyo.Set(initialize = data['flight'].unique())
flight_no = model.flight_no

In [9]:
# create set of DOW
model.DOW = pyo.Set(initialize = range(1,8), domain = PositiveIntegers)
DOW = model.DOW

In [10]:
# create set of hour
model.hour = pyo.Set(initialize = range(0,24), domain = NonNegativeIntegers)
hour = model.hour

In [11]:
# create set of airport
airport_from = data['from'].unique()
airport_to = data['to'].unique()
airport_set = set(np.concatenate((airport_from,airport_to)))
model.airport = pyo.Set(initialize = airport_set, ordered = False)
airport = model.airport

In [12]:
# Tạo 1 set để làm index các chuyến bay: số hiệu + AC + DOW
model.flight_index = pyo.Set(initialize = data['index'])
flight_index = model.flight_index

# 3. Parameters

# 4. Variables

In [13]:
model.assign_fleet = pyo.Var(flight_index, within = Binary, initialize = 0)
assign_fleet = model.assign_fleet

In [14]:
model.park_ac = pyo.Var(airport, DOW, hour, ac_type, domain = NonNegativeIntegers)
park_ac = model.park_ac

# 5. Constraints

## 5.1. Balance constraints

Tại mỗi mốc thời gian, số tàu đậu đỗ theo từng loại tàu của mỗi mốc thời gian + số tàu bay hạ cánh phải tương đương với số tàu đậu đỗ của mốc thời gian kế tiếp + số tàu bay cất cánh (Node1 + inwards = Node2 + outwards). Ngoài ra, để đảm bảo giả định sản phẩm tần suất các tuần giống nhau, time node cuối cùng trong tuần (23h Chủ Nhật) phải balance với node đầu tiên trong tuần (0h Thứ Hai).

In [15]:
def balance_constraint(model, a,d,h,ac):
    # khung giờ đầu tiên của thứ hai phải cân bằng với khung giờ cuối cùng của chủ nhật
    # (giả định sản phẩm tần suất của các tuần giống nhau)
    if h == 0 and d == 1:
        expr = (park_ac[a,d,h,ac] == 
                park_ac[a,7,23,ac]
                +sum(assign_fleet[i] for i in data[
                    (data['to'] == a)
                    &(data['DOW_ETA'] == 7)
                    &(data['ETA'].between(23,24,inclusive = 'left'))
                    &(data['AC'] == ac)]['index'])
                -sum(assign_fleet[i] for i in data[
                    (data['from'] == a)
                    &(data['DOW_ETD'] == 7)
                    &(data['ETD'].between(23,24,inclusive = 'left'))
                    &(data['AC'] == ac)]['index']))
    # khung giờ đầu tiên trong ngày = khung giờ cuối cùng ngày hôm trước:
    elif h == 0:
        expr = (park_ac[a,d,h,ac] == 
                park_ac[a,d-1,23,ac]
                +sum(assign_fleet[i] for i in data[
                    (data['to'] == a)
                    &(data['DOW_ETA'] == d-1)
                    &(data['ETA'].between(23,24,inclusive = 'left'))
                    &(data['AC'] == ac)]['index'])
                -sum(assign_fleet[i] for i in data[
                    (data['from'] == a)
                    &(data['DOW_ETD'] == d-1)
                    &(data['ETD'].between(23,24,inclusive = 'left'))
                    &(data['AC'] == ac)]['index']))
    # balance constraint cho các khung giờ trong 1 ngày:
    else:
        expr = (park_ac[a,d,h,ac] == 
                park_ac[a,d,h-1,ac]
                +sum(assign_fleet[i] for i in data[
                    (data['to'] == a)
                    &(data['DOW_ETA'] == d)
                    &(data['ETA'].between(h-1,h,inclusive = 'left'))
                    &(data['AC'] == ac)]['index'])
                -sum(assign_fleet[i] for i in data[
                    (data['from'] == a)
                    &(data['DOW_ETD'] == d)
                    &(data['ETD'].between(h-1,h,inclusive = 'left'))
                    &(data['AC'] == ac)]['index']))
    return expr

In [16]:
model.balance_constraint = pyo.Constraint(airport, DOW, hour, ac_type, rule = balance_constraint)

## 5.2. Coverage constraints

Mỗi 1 số hiệu chuyến bay nếu được lựa chọn khai thác thì chỉ được dùng duy nhất 1 loại tàu bay trong ngày. (Vd: VN087 không thể khai thác đồng thời 321 và 787 tại cùng 1 ngày).

In [17]:
def coverage_constraint(model, f,d):
    if data[(data['flight']==f)&(data['DOW_ETD']==d)].empty:
        expr = pyo.Constraint.Skip
    else:
        expr = sum(assign_fleet[i] for i in data[(data['flight']==f)&(data['DOW_ETD']==d)]['index']) <= 1
    return expr
# Rieeng các đường xuyên Đông Dương thì sẽ có 2 chuyến bay có chung 1 số hiệu chuyến bay trong 1 ngày, vì vậy phải thêm elif

In [18]:
model.coverage_constraint = pyo.Constraint(flight_no, DOW, rule = coverage_constraint)

## 5.3. Fleet constraints

Tổng số tàu bay tại các sân bay trong từng khung giờ không được lớn hơn số tàu bay đã thuê mua.

In [19]:
def fleet_constraint(model, d,h,ac):
    return pyo.inequality(0,sum(park_ac[a,d,h,ac] for a in airport),fleet[ac])
# có thể tính chi phí tàu bay cố định = sum(park_ac[a,d,h,ac] for a in airport)*chi phí 1 tàu

In [20]:
model.fleet_constraint = pyo.Constraint(DOW, hour, ac_type, rule = fleet_constraint)

## 5.4. Airport constraints

Một số đường bay chỉ dùng được tàu thân rộng, một số đường bay khác chỉ dùng được tàu AT7 do hạn chế về khoảng cách bay và cơ sở hạ tầng sân bay.

In [21]:
long_haul = ['CDG','DME','SVO','FRA','LHR','LAX','SFO','SYD','MEL']

In [22]:
def airport_constraint(model, i):
    return sum(assign_fleet[i] for i in data[(data['AC']=='A321')&(data['to'].isin(long_haul))]['index']) == 0

## 5.5. Product constraints

Để đảm bảo tính đồng nhất của sản phẩm, các chuyến bay trên 1 đường bay phải sử dụng loại tàu bay giống nhau giữa các ngày trong tuần.

In [23]:
# sum(flight for fleet a)*sum(flight for fleet b) == 0 => chỉ được chọn 1 trong 2 fleet a hoặc b cho tất cả tần suất trong tuần

In [24]:
def product_constraint(model,f):
    x = []
    for ac in ac_type:
        x.append(sum(assign_fleet[i] for i in data[(data['flight']==f)&(data['AC']==ac)]['index']))
    return np.prod(x) == 0

In [25]:
model.product_constraint = pyo.Constraint(flight_no, rule = product_constraint)

# 6. Objective function

In [26]:
model.obj = pyo.Objective(expr = (sum(assign_fleet[i]*data[data['index']==i]['revenue'].values[0] for i in flight_index)
                          -sum(assign_fleet[i]*data[data['index']==i]['cost'].values[0] for i in flight_index)), sense = pyo.maximize)

In [29]:
solver = SolverFactory('couenne', executable = "C:\\Users\\namtrantuan\\Desktop\\FAM\\bin\\couenne.exe")

In [ ]:
results = solver.solve(model)

# 7. Results

In [ ]:
model.pprint()

In [ ]:
results.solver.status

In [ ]:
results.solver.termination_condition

In [ ]:
optimal_values = [value(assign_fleet[i]) for i in assign_fleet]

In [ ]:
park_ac_data = {(a,d,h,ac, v.name): value(v) for (a,d,h,ac), v in park_ac.items()}
df_park_ac = pd.DataFrame.from_dict(park_ac_data, orient="index", columns=["variable value"])
df_park_ac.reset_index(inplace = True)
print(df_park_ac)

In [ ]:
df_assign_fleet = pd.DataFrame.from_dict(assign_fleet.extract_values(), orient='index', columns=[str(assign_fleet)])
df_assign_fleet.reset_index(inplace = True)
print(df_assign_fleet)

In [ ]:
result = data.merge(df_assign_fleet, on = 'index')

In [ ]:
result.to_excel('result.xlsx')